In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import torch
import torchaudio as ta
import torchaudio.functional as taf
import torchaudio.transforms as tat
from torchvision import transforms

print(torch.__version__)        
print(ta.__version__)

import matplotlib
import matplotlib.pyplot as plt
from IPython.display import Audio, display

import pandas as pd
import os
import pprint

from typing import *
import itertools
from collections import Counter

import numpy as np
from datetime import datetime
from lark.config import Config
from lark.learner import Learner
from lark.ops import Sig2Spec, MixedSig2Spec

1.8.1
0.8.0a0+e4e171a


In [61]:
# cfg = Config(
#     sites=['SSW'], 
#     use_neptune=True, 
#     n_epochs=15, 
#     bs=32, 
#     lr=1e-3, 
#     model='tf_efficientnet_b0_ns',
#     scheduler='torch.optim.lr_scheduler.CosineAnnealingLR',
#     loss_fn='lark.ops.SigmoidFocalLossStar',
#     use_pink_noise=0,
#     use_recorded_noise=0.5,
#     use_overlays=True,
#     apply_filter=0,
# )

cfg = Config(
        sites=None,
        use_neptune=True,
        n_epochs=12,
        bs=32,
        n_samples_per_label=1000,
        lr=1e-3,
        model='tf_efficientnet_b0_ns',
        scheduler='torch.optim.lr_scheduler.CosineAnnealingLR',
        # scheduler='torch.optim.lr_scheduler.OneCycleLR',
        loss_fn='lark.ops.SigmoidFocalLossStar',
        use_pink_noise=0.5,
        use_recorded_noise=0.5,
        use_overlays=True,
        apply_filter=0.2,
    )

In [72]:
cfg.as_dict()

{'sites': None,
 'data_dir': 'data/birdclef-2021',
 'checkpoint_dir': 'checkpoints',
 'bs': 32,
 'n_samples_per_label': 1000,
 'n_workers': 12,
 'train_duration': 5,
 'valid_duration': 5,
 'seed': 231,
 'use_pink_noise': 0.5,
 'use_recorded_noise': 0.5,
 'use_secondary_labels': False,
 'noise_nsr_dbs': [20, 10, 3],
 'pink_noise_nsr_dbs': [10, 5, 3],
 'noise_dir': 'data/noise/BirdVox-DCASE-20k/wav-32k',
 'use_overlays': True,
 'max_overlays': 5,
 'overlay_weights': [0.71986223,
  0.21010333,
  0.06314581,
  0.00574053,
  0.00114811],
 'overlay_snr_dbs': [20, 10, 3],
 'apply_filter': 0.2,
 'use_neptune': True,
 'log_batch_metrics': False,
 'sr': 32000,
 'n_fft': 512,
 'window_length': 512,
 'n_mels': 64,
 'hop_length': 312,
 'f_min': 150,
 'f_max': 15000,
 'f1_threshold': 0.5,
 'lr': 0.001,
 'n_epochs': 12,
 'model': 'tf_efficientnet_b0_ns',
 'optimizer': 'torch.optim.Adam',
 'loss_fn': 'lark.ops.SigmoidFocalLossStar',
 'scheduler': 'torch.optim.lr_scheduler.CosineAnnealingLR',
 'labels'

In [62]:
import gc
gc.collect()
torch.cuda.empty_cache()


In [63]:
import timm
from pprint import pprint
model_names = timm.list_models(pretrained=True)
# model_names

In [64]:
prep = MixedSig2Spec(cfg)

# prep = Sig2Spec(cfg)

# main_model = torch.hub.load('zhanghang1989/ResNeSt', 'resnest50', pretrained=True)

main_model = timm.create_model('tf_efficientnet_b0_ns', pretrained=True)


posp = torch.nn.Sequential(
#     torch.nn.Linear(in_features=2048, out_features=1024, bias=True),
#     torch.nn.Dropout(p=0.2),
#     torch.nn.ReLU(),
#     torch.nn.Linear(in_features=1024, out_features=512, bias=True),
#     torch.nn.Dropout(p=0.2),
#     torch.nn.ReLU(),
    torch.nn.Linear(in_features=1280, out_features=len(cfg.labels), bias=True),
)
# main_model.fc = posp
main_model.classifier = posp

In [65]:

# for layer in [main_model.conv_stem, main_model.bn1, main_model.act1, main_model.blocks[0], main_model.blocks[1]]:
#     for param in layer.parameters():
#         param.requires_grad = False


In [66]:
model = torch.nn.Sequential(prep, main_model)
model = model.cuda()

In [55]:
# model[1]

In [67]:
lrn = Learner("tf_efficientnet_b0_ns-full", cfg, model)

tf_efficientnet_b0_ns-full-20210525-181231


In [68]:
lrn.learn()

https://ui.neptune.ai/botkop/lark/e/LAR-187


  0%|          | 0/12 [00:00<?, ?it/s]

  0%|          | 0/12407 [00:00<?, ?it/s]

  0%|          | 0/75 [00:00<?, ?it/s]

2021-05-25 18:44:13 epoch:   1 train loss: 0.002683 train f1: 0.144946 valid loss: 0.001151 valid f1: 0.024370


...

2021-05-26 00:00:45 epoch:  11 train loss: 0.001440 train f1: 0.693066 valid loss: 0.001173 valid f1: 0.100380


  0%|          | 0/12407 [00:00<?, ?it/s]

  0%|          | 0/75 [00:00<?, ?it/s]

2021-05-26 00:32:25 epoch:  12 train loss: 0.001437 train f1: 0.693398 valid loss: 0.001173 valid f1: 0.118797


Unexpected error in ping thread.
Traceback (most recent call last):
  File "/home/koen/apps/anaconda3/envs/lark/lib/python3.8/site-packages/urllib3/connectionpool.py", line 699, in urlopen
    httplib_response = self._make_request(
  File "/home/koen/apps/anaconda3/envs/lark/lib/python3.8/site-packages/urllib3/connectionpool.py", line 445, in _make_request
    six.raise_from(e, None)
  File "<string>", line 3, in raise_from
  File "/home/koen/apps/anaconda3/envs/lark/lib/python3.8/site-packages/urllib3/connectionpool.py", line 440, in _make_request
    httplib_response = conn.getresponse()
  File "/home/koen/apps/anaconda3/envs/lark/lib/python3.8/http/client.py", line 1347, in getresponse
    response.begin()
  File "/home/koen/apps/anaconda3/envs/lark/lib/python3.8/http/client.py", line 307, in begin
    version, status, reason = self._read_status()
  File "/home/koen/apps/anaconda3/envs/lark/lib/python3.8/http/client.py", line 268, in _read_status
    line = str(self.fp.readline(_MAX

In [69]:
lrn.evaluate()

  0%|          | 0/75 [00:00<?, ?it/s]

,thresh,tp,tn,fp,fn,f1
0,0.0,1183,0,951617,0,0.002480
1,0.1,483,939820,11797,700,0.071752
2,0.2,263,950238,1379,920,0.186195
3,0.3,162,951277,340,1021,0.192285
4,0.4,111,951466,151,1072,0.153633
5,0.5,79,951549,68,1104,0.118797
6,0.6,47,951582,35,1136,0.074308
7,0.7,30,951596,21,1153,0.048622
8,0.8,17,951605,12,1166,0.028053
9,0.9,5,951613,4,1178,0.008389


In [70]:
lrn.load_checkpoint('best')

{'epoch': 2,
 'valid_loss': 0.0011455502287329486,
 'valid_score': 0.05832693725824356}

In [71]:
lrn.evaluate()

  0%|          | 0/75 [00:00<?, ?it/s]

,thresh,tp,tn,fp,fn,f1
0,0.0,1183,0,951617,0,0.002480
1,0.1,451,944322,7295,732,0.101019
2,0.2,211,950730,887,972,0.185007
3,0.3,113,951288,329,1070,0.139077
4,0.4,61,951446,171,1122,0.086219
5,0.5,38,951535,82,1145,0.058327
6,0.6,20,951582,35,1163,0.032310
7,0.7,8,951601,16,1175,0.013256
8,0.8,5,951605,12,1178,0.008333
9,0.9,2,951609,8,1181,0.003353


In [13]:
lrn = Learner("tf_efficientnet_b0_ns-focal-loss", cfg, model)

tf_efficientnet_b0_ns-focal-loss-20210525-153516


In [14]:
lrn.learn()

  0%|          | 0/12 [00:00<?, ?it/s]

  0%|          | 0/875 [00:00<?, ?it/s]

  0%|          | 0/38 [00:00<?, ?it/s]

2021-05-25 15:37:37 epoch:   1 train loss: 0.026205 train f1: 0.214321 valid loss: 0.015295 valid f1: 0.060226


  0%|          | 0/875 [00:00<?, ?it/s]

...

  0%|          | 0/38 [00:00<?, ?it/s]

2021-05-25 16:01:06 epoch:  11 train loss: 0.013343 train f1: 0.767252 valid loss: 0.014022 valid f1: 0.260394


  0%|          | 0/875 [00:00<?, ?it/s]

  0%|          | 0/38 [00:00<?, ?it/s]

2021-05-25 16:03:27 epoch:  12 train loss: 0.013407 train f1: 0.770511 valid loss: 0.013902 valid f1: 0.257143


In [15]:
lrn.evaluate()

  0%|          | 0/38 [00:00<?, ?it/s]

,thresh,tp,tn,fp,fn,f1
0,0.0,770,0,32830,0,0.044807
1,0.1,553,23627,9203,217,0.105073
2,0.2,331,31988,842,439,0.340710
3,0.3,229,32671,159,541,0.395510
4,0.4,155,32777,53,615,0.316973
5,0.5,117,32807,23,653,0.257143
6,0.6,93,32818,12,677,0.212571
7,0.7,67,32825,5,703,0.159145
8,0.8,50,32829,1,720,0.121803
9,0.9,21,32830,0,749,0.053097


In [16]:
lrn.load_checkpoint('best')

{'epoch': 8,
 'valid_loss': 0.01348914288131422,
 'valid_score': 0.25749167799949646}

In [17]:
lrn.evaluate()

  0%|          | 0/38 [00:00<?, ?it/s]

,thresh,tp,tn,fp,fn,f1
0,0.0,770,0,32830,0,0.044807
1,0.1,526,24822,8008,244,0.113070
2,0.2,326,32202,628,444,0.378190
3,0.3,219,32718,112,551,0.397820
4,0.4,154,32791,39,616,0.319834
5,0.5,116,32815,15,654,0.257492
6,0.6,91,32820,10,679,0.208955
7,0.7,60,32827,3,710,0.144058
8,0.8,38,32828,2,732,0.093827
9,0.9,21,32830,0,749,0.053097


In [51]:
lrn.load_checkpoint('latest')

{'epoch': 14,
 'valid_loss': 0.0097759227378686,
 'valid_score': 0.33207547664642334}

In [52]:
lrn.scheduler.state_dict()

{'T_max': 10,
 'eta_min': 1e-05,
 'base_lrs': [0.0001],
 'last_epoch': 15,
 '_step_count': 16,
 'verbose': False,
 '_get_lr_called_within_step': False,
 '_last_lr': [5.5000000000000205e-05]}

In [53]:
lrn.optimizer.state_dict

<bound method Optimizer.state_dict of Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    initial_lr: 0.0001
    lr: 5.5000000000000205e-05
    weight_decay: 0
)>

# hyperpar scan

In [90]:

def default_cfg():
    return Config(
        sites=['SSW'], 
        use_neptune=True, 
        n_epochs=30, 
        bs=32, 
        lr=1e-3,
        model='resnest50-hyper-scan',
        scheduler='torch.optim.lr_scheduler.CosineAnnealingLR',
        loss_fn='lark.ops.FocalLoss',
        use_pink_noise=0,
        use_recorded_noise=False,
        use_overlays=False,
        apply_filter=0,
    )


par_dict = dict(
    use_pink_noise=0.5,
    use_recorded_noise=0.5,
    use_overlays=True,
    apply_filter=0.5,    
)

def apply_par(k):
    cfg = default_cfg()
    setattr(cfg, k, par_dict[k])
    return cfg

configs = [apply_par(k) for k in par_dict]
    

In [ ]:
def make_model():
    prep = MixedSig2Spec(cfg)
    main_model = torch.hub.load('zhanghang1989/ResNeSt', 'resnest50', pretrained=True)
    posp = torch.nn.Sequential(
        torch.nn.Linear(in_features=2048, out_features=1024, bias=True),
        torch.nn.Dropout(p=0.2),
        torch.nn.ReLU(),
        torch.nn.Linear(in_features=1024, out_features=512, bias=True),
        torch.nn.Dropout(p=0.2),
        torch.nn.ReLU(),
        torch.nn.Linear(in_features=512, out_features=len(cfg.labels), bias=True),
    )
    main_model.fc = posp
    model = torch.nn.Sequential(prep, main_model)
    model = model.cuda()
    return model


Using cache found in /home/koen/.cache/torch/hub/zhanghang1989_ResNeSt_master


In [69]:
for cfg in configs:
    model = make_model()
    lrn = Learner("resnest50-param-scan", cfg, model)
    lrn.learn()

resnest50-focal-loss-20210521-193723
